In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json

In [2]:
baseurl = 'https://bina.az'
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
}

for p in range(1, 2):
    r = requests.get(f'https://bina.az/alqi-satqi?page={p}', headers = headers)
    soup = BeautifulSoup(r.content, 'lxml')
    productlist = soup.find_all('div', class_ = 'items-i')
    
    productlinks = []
    
    products = {
    'Location' : [],
    'Category' : [],
    'Floor' : [],
    'Area' : [],
    'GroundArea' : [],
    'Rooms' : [],
    'TitleDeed' : [],
    'Mortgage' : [],
    'Repair' : [],
    'Phone' : [],
    'Price' : []
    }
    
    for item in productlist:
        links = item.find_all('a', class_ = 'item_link', href = True)
        productlinks.append(baseurl + links[0]['href'])
        
    for link in productlinks:
        r = requests.get(link, headers = headers)
        soup = BeautifulSoup(r.content, 'lxml')
        properties = soup.find_all('div', class_ = 'product-properties__i')
        
        price_text = soup.find('span', class_ = 'price-val').get_text(strip = True)
        products['Price'].append(price_text)
        
        loc_text = soup.find('div', class_ = 'product-map__left__address').get_text(strip = True)
        products['Location'].append(loc_text)
        
        req = requests.get(f'{link}/phones', headers = headers)
        sp = BeautifulSoup(req.content, 'lxml')
        phone_json = sp.find('p').get_text(strip = True)
        phone_text = json.loads(phone_json).get('phones', [])
        products['Phone'].append(phone_text)
        
        labels = {}        
        for property in properties:
            lbl = property.find('label', class_ = 'product-properties__i-name').get_text(strip = True)
            spn = property.find('span', class_ = 'product-properties__i-value').get_text(strip = True)
            labels[lbl] = spn
            
        products['Category'].append(labels.get('Kateqoriya', 'null'))
        products['Floor'].append(labels.get('Mərtəbə', 'null'))
        products['Area'].append(labels.get('Sahə', 'null'))
        products['Rooms'].append(labels.get('Otaq sayı', 'null'))
        products['TitleDeed'].append(labels.get('Çıxarış', 'null'))
        products['Mortgage'].append(labels.get('İpoteka', 'null'))
        products['Repair'].append(labels.get('Təmir', 'null'))
        products['GroundArea'].append(labels.get('Torpaq sahəsi', 'null'))
        
    df = pd.DataFrame(products)
df

,Location,Category,Floor,Area,GroundArea,Rooms,TitleDeed,Mortgage,Repair,Phone,Price
0,İbrahimpaşa Dadaşov küçəsi,Obyekt,null,344 m²,null,null,yoxdur,null,var,[(050) 241-11-12],1 480 000
1,Azadlıq prospekti 136,Köhnə tikili,4 / 5,115 m²,null,3,var,null,var,[(050) 624-31-33],215 000
2,Binəqədi qəsəbəsi.,Obyekt,null,180 m²,null,null,yoxdur,null,var,[(077) 377-27-87],800 000
3,Salatın Əsgərova küçəsi,Torpaq,null,31 sot,null,null,var,null,null,[(055) 218-63-43],9 000
4,Zərdabi prospekti 92A,Yeni tikili,15 / 17,70 m²,null,2,var,null,var,[(077) 444-55-66],174 000
5,Şüvəlan qəsəbəsi,Həyət evi/Bağ evi,null,162 m²,4 sot,4,var,null,var,[(050) 210-15-02],219 000
6,Naxçıvan küçəsi,Yeni tikili,9 / 16,135 m²,null,3,var,null,var,[(050) 209-37-40],310 000
7,Rüstəm Rüstəmli küç.,Obyekt,null,330 m²,null,null,yoxdur,null,null,[(050) 673-73-45],620 000
8,Qara Qarayev pr.,Yeni tikili,5 / 17,60 m²,null,3,yoxdur,null,var,[(010) 232-82-83],135 000
9,Z.Xəlilov 26,Köhnə tikili,1 / 5,60 m²,null,2,var,null,var,[(050) 209-37-38],120 000
